# Import library yang dibutuhkan

In [3]:
#proses data
import numpy as np 
import pandas as pd

#Text Preprosessing
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
import nltk
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder

#Models evaluasi
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix, accuracy_score, classification_report    
from sklearn.model_selection import StratifiedKFold

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\RanggaSaputra\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# load data

In [4]:
data = pd.read_csv('./archive/data job posts.csv')
data.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


# Cek info data

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19001 entries, 0 to 19000
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   jobpost           19001 non-null  object
 1   date              19001 non-null  object
 2   Title             18973 non-null  object
 3   Company           18994 non-null  object
 4   AnnouncementCode  1208 non-null   object
 5   Term              7676 non-null   object
 6   Eligibility       4930 non-null   object
 7   Audience          640 non-null    object
 8   StartDate         9675 non-null   object
 9   Duration          10798 non-null  object
 10  Location          18969 non-null  object
 11  JobDescription    15109 non-null  object
 12  JobRequirment     16479 non-null  object
 13  RequiredQual      18517 non-null  object
 14  Salary            9622 non-null   object
 15  ApplicationP      18941 non-null  object
 16  OpeningDate       18295 non-null  object
 17  Deadline    

# Pilih fitur yang digunakan

In [6]:
#Mengambil data IT saja
df = data[data['IT']]

#Menggunakan kolom 'RequiredQual', 'Eligibility', 'Title', 'JobDescription', 'JobRequirment'
cols = ['RequiredQual', 'Eligibility', 'Title', 'JobDescription', 'JobRequirment']
df = df[cols]
df.head()

,RequiredQual,Eligibility,Title,JobDescription,JobRequirment
4,- University degree; economical background is ...,NaN,Software Developer,NaN,- Rendering technical assistance to Database M...
15,"- Excellent knowledge of Windows 2000 Server, ...",NaN,Network Administrator,NaN,- Network monitoring and administration;\r\n- ...
19,"As a GD you are creative, innovative and have\...",NaN,Graphic Designer,The position of Graphic Designer (GD) demands ...,Graphic Designer will be responsible for every...
34,Participants should be mid-level professionals...,NaN,Demographic Analysis Workshop,NaN,NaN
35,- Work experience of at least two years; \r\n-...,NaN,Programmer,NaN,NaN


In [7]:
#Pilih 21 pekerjaan teratas untuk mengatasi imbalance
kelas = df['Title'].value_counts()[:21]
keys = kelas.keys().to_list()

df = df[df['Title'].isin(keys)]
df['Title'].value_counts()

Software Developer           134
Web Developer                101
Java Developer                88
Graphic Designer              75
Software Engineer             69
Senior Java Developer         69
PHP Developer                 65
Senior Software Engineer      63
Programmer                    56
IT Specialist                 55
Senior QA Engineer            43
Senior Software Developer     41
Android Developer             37
.NET Developer                36
Senior .NET Developer         34
Senior PHP Developer          34
iOS Developer                 31
Software QA Engineer          29
Senior Web Developer          29
Database Developer            29
Database Administrator        28
Name: Title, dtype: int64

In [8]:
#Fungsi untuk menyamakan semua data
def Changge_Titles(x):
    x = x.strip()
    if x == 'Senior Java Developer':
        return 'Java Developer'
    elif x == 'Senior Software Engineer':
        return 'Software Engineer'
    elif x == 'Senior QA Engineer':
        return 'Software QA Engineer'
    elif x == 'Senior Software Developer':
        return 'Senior Web Developer'
    elif x =='Senior PHP Developer':
        return 'PHP Developer'
    elif x == 'Senior .NET Developer':
        return '.NET Developer'
    elif x == 'Senior Web Developer':
        return 'Web Developer'
    elif x == 'Database Administrator':
        return 'Database Admin/Dev'
    elif x == 'Database Developer':
        return 'Database Admin/Dev'
    else:
        return x
        
    
df['Title'] = df['Title'].apply(Changge_Titles)
df['Title'].value_counts()

Java Developer          157
Software Developer      134
Software Engineer       132
Web Developer           130
PHP Developer            99
Graphic Designer         75
Software QA Engineer     72
.NET Developer           70
Database Admin/Dev       57
Programmer               56
IT Specialist            55
Senior Web Developer     41
Android Developer        37
iOS Developer            31
Name: Title, dtype: int64

# Teks Preprosessing

In [9]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        # Mengubah kata ke dalam kata dasarnya kembali 
        self.wnl = WordNetLemmatizer()
        # membuata stopwords list 
        self.stopwords = stopwords.words('english')
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.stopwords]

# menghapus baris baru dan karakter tertentu
df['RequiredQual']=df['RequiredQual'].apply(lambda x: x.replace('\n', ' ').replace('\r', '').replace('- ', ''). replace(' - ', ' to '))

# Pembobotan

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
# pisahkan antara label dan fitur 
y = df['Title']
X = df['RequiredQual']

# pembobotan menggunakan tf-idf 
vectorizer = TfidfVectorizer(tokenizer=LemmaTokenizer(), stop_words='english')
vectorizer.fit(X)

# tranformasi teks ke tf-idf fitur
tfidf_matrix = vectorizer.transform(X)

# menubah dalam bentuk array
X_tdif = tfidf_matrix.toarray()

# encoding label 
enc = LabelEncoder() 
enc.fit(y.values)
y_enc=enc.transform(y.values)

# pisahkan train dan test
X_train_words, X_test_words, y_train, y_test = train_test_split(X, y_enc, test_size=0.15, random_state=10)

X_train = vectorizer.transform(X_train_words)
X_train = X_train.toarray()

X_test = vectorizer.transform(X_test_words)
X_test = X_test.toarray()

c:\Users\RanggaSaputra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\RanggaSaputra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn(


## Build Model

In [16]:
# atur parameter yang akan dipakai
param_grid = {
    'verbose': [1, 2, 3],
    'max_iter': [25, 50, 75, 100],
    'C': [0.1, 0.25, 0.5, 0.75, 1.0],
    'penalty': ['l1', 'l2', 'elasticnet', 'None'],
    'solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
    'multi_class': ['auto', 'ovr', 'multinomial']
}

# Buat objek GridSearchCV
grid_search = GridSearchCV(logistic, param_grid, cv=5, n_jobs=-1)

# Lakukan proses hyperparameter tuning
grid_search.fit(X_train, y_train)

# Hasil dari hyperparameter tuning
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Hyperparameters: ", best_params)
print("Best Cross-validation Score: {:.2f}".format(best_score))

# Evaluasi model terbaik pada data uji
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy of Best Model: {:.2f}".format(test_accuracy))

c:\Users\RanggaSaputra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
3600 fits failed out of a total of 7200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3600 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\RanggaSaputra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\RanggaSaputra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Users\Ran

Best Hyperparameters:  {'C': 0.1, 'max_iter': 25, 'multi_class': 'auto', 'penalty': 'l2', 'solver': 'lbfgs', 'verbose': 1}
Best Cross-validation Score: nan
Test Accuracy of Best Model: 0.44


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


# Model evaluasi

In [11]:
# gunakan model dengan parameter terbaik
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(C= 1.0, max_iter= 25, multi_class= 'auto', penalty= 'l2', solver= 'lbfgs', verbose= 1)
train_preds = logistic.fit(X_train, y_train).predict(X_train)
test_preds = logistic.predict(X_test)

print('Train acc: {0}'.format(accuracy_score(y_train, train_preds)))
print('Test acc: {0}'.format(accuracy_score(y_test, test_preds)))

print('classification report')
print(classification_report(y_test, test_preds))

print(f'confusion matrix:\n {confusion_matrix(y_test, test_preds)}')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Train acc: 0.9014373716632443
Test acc: 0.7616279069767442
classification report
              precision    recall  f1-score   support

           0       0.83      0.83      0.83        12
           1       1.00      1.00      1.00         5
           2       1.00      0.50      0.67         6
           3       1.00      1.00      1.00        13
           4       0.89      0.80      0.84        10
           5       0.75      0.90      0.82        20
           6       0.80      0.86      0.83        14
           7       0.89      0.80      0.84        10
           8       1.00      0.17      0.29         6
           9       0.52      0.65      0.58        23
          10       0.76      0.57      0.65        23
          11       1.00      1.00      1.00         7
          12       0.60      0.79      0.68        19
          13       1.00      0.75      0.86         4

    accuracy                           0.76       172
   macro avg       0.86      0.76      0.78       172

c:\Users\RanggaSaputra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


# Hasil rekomendasi

In [12]:
# dictionary yang akan digunakan untuk menyimpan hasil-hasil yang diperoleh dari prediksi
preds_data = {'Current Position Requirments': [], 'Current Position': [], 'Alternative 1': [], 'Alternative 2': []}

# model yang sebelumnya dilatih
y_preds_proba = logistic.predict_proba(X_test)

counter = 0 
for idx, (pred_row, true_job_position) in enumerate(zip(y_preds_proba, y_test)):
    # urutkan probabilitas prediksi dalam urutan dari terendah ke tertinggi
    class_preds = np.argsort(pred_row)
    # menghapus label sebenarnya
    for i in [-1, -2]:
        if class_preds[i] == true_job_position:
            class_preds=np.delete(class_preds,i)
    # mendapatkan 2 label tertinggi         
    top_classes = class_preds[-2:]
    # mengembalikan nama label ke sebenarnya 
    class_names = enc.inverse_transform(top_classes)
    true_job_position_name = enc.inverse_transform([true_job_position])
    # menyimpan dictionary
    preds_data['Current Position Requirments'].append(X_test_words.iloc[idx])
    preds_data['Current Position'].append(true_job_position_name[0])
    preds_data['Alternative 1'].append(class_names[1])
    preds_data['Alternative 2'].append(class_names[0])

    counter +=1

In [15]:
preds_df = pd.DataFrame.from_dict(preds_data)
preds_df.to_csv('hasil_rekomendasi.csv', index=False)

In [16]:
preds_df

,Current Position Requirments,Current Position,Alternative 1,Alternative 2
0,From 1 to 5 years of experience in Android dev...,Android Developer,iOS Developer,Software Engineer
1,University degree in the appropriate field of ...,PHP Developer,Web Developer,Software Developer
2,At least 3 years of experience with PHP and Ja...,Web Developer,Software Engineer,PHP Developer
3,At least 3 years of work experience as a Java ...,Java Developer,Software Engineer,Software Developer
4,Bachelor??s degree in Engineering or Informati...,Software Engineer,Software Developer,Web Developer
...,...,...,...,...
167,"Artistic ability to draw, sketch and design cr...",Graphic Designer,Web Developer,Software Developer
168,"BS in Computer Science, Maths or a related dis...",Software Engineer,Software Developer,PHP Developer
169,University degree in Information Technology or...,IT Specialist,Software Developer,Database Admin/Dev
170,"University degree in Computer Science, Informa...",Software Developer,Software Engineer,Web Developer
